In [52]:
import pandas as pd
import numpy as np
from acquire import get_titanic_data
from prepare import clean_titanic
import warnings
warnings.filterwarnings('ignore')

In [53]:
# assigning train, validate, and test 
# to the output of clean_titanic
# which is being calld on the return
# of the invocation of get_titanic_data()
train, val, test = clean_titanic(get_titanic_data())

Create a new notebook, knn_model, and work with the titanic dataset to answer the following:

Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [54]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [55]:
# make the sklearn knn object
clf = KNN()

In [59]:
X_cols = train.columns.to_list()
y_col = 'survived'

In [60]:
# drop out the values we def dont want in X currently:
# 1: our target:
X_cols.remove('survived')
# 2: nonencoded features (not nice to these types of algorithms)
X_cols.remove('sex')
X_cols.remove('embark_town')

In [65]:
train[y_col].mode().values

array([0])

In [71]:
# baseline value of zero
# for the same number of values that exist
# within the samples in our training set
baseline_preds = [0 for i in range(0, len(train))]

In [70]:
# verify that our range of predictions is the same as 
# the length of the dataframe itself:
len([0 for i in range(0, len(train))]) == len(train)

True

In [74]:
# establish that baseline!
baseline_accuracy = (train[y_col] == 0).mean()

In [75]:
# fit the model:
# let's define an X and a y for our needs:
X_train = train[X_cols]
y_train = train[y_col]

In [77]:
# I can still reference a subset of X_train with
# a list of column names as long as they live inside
# of X_train!
# X_train[['age','pclass']]

In [78]:
# fit the thing / train the model
clf.fit(X_train, y_train)

KNeighborsClassifier()

Evaluate your results using the model score, confusion matrix, and classification report.

In [79]:
from sklearn.metrics import accuracy_score, \
confusion_matrix, \
classification_report

In [81]:
# model score
model_score = clf.score(X_train, y_train)
# alternatively:
knn_preds = clf.predict(X_train)
# Theyre the same thing!!
accuracy = accuracy_score(y_train, knn_preds)
print(f' model score: {model_score}, accuracy score: {accuracy}')
# confusion matrix
confusion_matrix(y_train, knn_preds)

In [114]:
pd.crosstab(
y_train,pd.Series(
    knn_preds, name='KNN_preds', index=y_train.index))

KNN_preds,0,1
survived,,
0,264,43
1,48,143


In [87]:
# classification report:
print(classification_report(y_train, knn_preds))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       307
           1       0.77      0.75      0.76       191

    accuracy                           0.82       498
   macro avg       0.81      0.80      0.81       498
weighted avg       0.82      0.82      0.82       498



Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [88]:
def get_metrics(y, y_pred):
    TN, FP, FN, TP = confusion_matrix(y, y_pred).ravel()
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    TPR = recall = TP / (TP + FN)
    FPR = FP / (FP + TN)
    TNR = TN / (FP + TN)
    FNR = FN / (FN + TP)
    precision =  TP / (TP + FP)
    f1 =  2 * ((precision * recall) / ( precision + recall))
    support_0 = train[train.survived == 0].shape[0]
    support_1 = train[train.survived == 1].shape[0]
    print(f'''
    Accuracy: {accuracy}
    True Positive Rate: {TPR}
    True Negative Rate: {TNR}
    False Positive Rate: {FPR}
    False Negative Rate: {FNR}
    Precision : {precision}
    f1 score: {f1}
    supports: 0: {support_0} 1: {support_1}
    ''')

In [89]:
get_metrics(y_train, knn_preds)


    Accuracy: 0.8172690763052208
    True Positive Rate: 0.7486910994764397
    True Negative Rate: 0.8599348534201955
    False Positive Rate: 0.14006514657980457
    False Negative Rate: 0.2513089005235602
    Precision : 0.7688172043010753
    f1 score: 0.7586206896551724
    supports: 0: 307 1: 191
    


Run through steps 1-3 setting k to 10

In [91]:
clf2 = KNN(n_neighbors=10)
clf2.fit(X_train, y_train)
clf2.score(X_train, y_train)

0.7710843373493976

Run through steps 1-3 setting k to 20

In [95]:
clf3 = KNN(n_neighbors=20)
clf3.fit(X_train, y_train)
clf3.score(X_train, y_train)

0.7409638554216867

What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [ ]:
# So far, the first model performs best on in-sample data:
# it has a greater accuracy based on the 5 closest neighbors
# we will see that the further out we reach, the more
# potentially dissimilar the inidiviual points get to the 
# one that we are trying to predict
# we dont want to cast too wide of a net,
# and we want to get the best fit for the number of neighbors
# which varies based on the data!
# which is why you change it!
# for the data!
# data!

Which model performs best on our out-of-sample data from validate?

In [96]:
# let's continue that thought after the logreg

--------

Create a new notebook, logistic_regression, use it to answer the following questions:

Create a model that includes only age, fare, and pclass. Does this model perform better than your baseline?

In [97]:
subset = ['age', 'fare', 'pclass']
X_train[subset]

,age,fare,pclass
474,22.00000,9.8375,3
370,25.00000,55.4417,1
573,30.53944,7.7500,3
110,47.00000,52.0000,1
167,45.00000,27.9000,3
...,...,...,...
735,28.50000,16.1000,3
163,17.00000,8.6625,3
770,24.00000,9.5000,3
196,30.53944,7.7500,3


In [98]:
from sklearn.linear_model import LogisticRegression

In [101]:
# make our thing:
lr = LogisticRegression()
lr.fit(X_train[subset], y_train)
lr.score(X_train[subset], y_train)

0.7449799196787149

Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

In [103]:
subset.append('sex_male')

In [104]:
lr2 = LogisticRegression()
lr2.fit(X_train[subset], y_train)
lr2.score(X_train[subset], y_train)

0.7931726907630522

Try out other combinations of features and models.

In [105]:
lr2.coef_

array([[-3.40040812e-02,  1.65997921e-03, -1.19645022e+00,
        -2.47611775e+00]])

In [ ]:
# we would likely not do this at random:
# why?
# computational expense!
# lack of insight!
# We will rely on our exploration to provide meaningful
# combinations of features at this phase
# consider ranking your perception of most valuable features
# take into account stats testing

Use you best 3 models to predict and evaluate on your validate sample.

In [107]:
# validate:
X_val, y_val = val[X_cols], val[y_col]

In [108]:
# validate scores:
# always remember to predict, and not re-fit

In [109]:
print(f'''
KNN model 1 validate score: {clf.score(X_val, y_val)}
KNN model 2 validate score: {clf2.score(X_val, y_val)}
logreg model 2 validate score: {lr2.score(X_val[subset], y_val)}''')


KNN model 1 validate score: 0.6401869158878505
KNN model 2 validate score: 0.6588785046728972
logreg model 2 validate score: 0.7336448598130841


Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

In [110]:
# if i were to finish here,
# i would pick lr2 as my best model
# and use it once on test to conclude my pipeline:
lr2.score(test[subset], test[y_col])

0.8268156424581006